In [51]:
#Imports e configuração de caminho
import sys, os
import pandas as pd
import numpy as np

# Adiciona src/ ao path para importar modulos
src_path = os.path.join(os.getcwd(), "src")
sys.path.append(src_path)

from fetchers import fetch_inmet_climate, fetch_ibge_population_density
from src.utils import normalize_column_names

# Cria pastas de saída
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)

In [52]:
#Carregamento da base de dados
df_dengue = pd.read_csv("https://raw.githubusercontent.com/valdineyatilio/ProjetoAplicado-IV/main/A1/DadosDengueCSV.csv")
df_dengue = normalize_column_names(df_dengue)
df_dengue.to_csv("data/raw/dengue.csv", index=False)

print("Base de dengue carregada:", df_dengue.shape)
df_dengue.to_csv("data/raw/dengue.csv", index=False)

Base de dengue carregada: (812, 30)


In [53]:
#Coleta dados climáticos do INMET ou gera simulação
stations = ["A507", "A518", "A541"] 
try:
    df_clima = fetch_inmet_climate(stations, "2015-01-01", "2025-12-31")
    if df_clima.empty:
        raise RuntimeError("Dados vazios")
    print("Clima INMET coletado:", df_clima.shape)
except Exception:
    print("[WARN] Nenhuma estação INMET retornou dados válidos. Gerando dados simulados.")

    weeks = pd.date_range("2018-01-01", "2023-12-31", freq="W-MON")
    df_clima = pd.DataFrame({
        "datahora": weeks,
        "station": "SIMULADA",
        "year": weeks.isocalendar().year,
        "week": weeks.isocalendar().week,
        "t_mean": np.random.normal(loc=22, scale=3, size=len(weeks)),
        "precip_sum": np.random.gamma(shape=2, scale=10, size=len(weeks))
    })

df_clima = normalize_column_names(df_clima)
df_clima.to_csv("data/raw/clima_inmet.csv", index=False)


[WARN] Estação A507 retornou status 404. Pulando.
[WARN] Estação A518 retornou status 404. Pulando.
[WARN] Estação A541 retornou status 404. Pulando.
[WARN] Nenhuma estação retornou dados válidos.
[WARN] Nenhuma estação INMET retornou dados válidos. Gerando dados simulados.


In [55]:
#Coleta densidade populacional do IBGE ou gera simulação
if "localidade_id" in df_dengue.columns:
    mun_ids = df_dengue["Localidade_id"].dropna().unique().astype(int).tolist()
else:
    raise ValueError("Coluna 'Localidade_id' não encontrada na base de dengue.")

try:
    df_pop = fetch_ibge_population_density(mun_ids, 2020)
    if df_pop.empty:
        raise RuntimeError("Dados vazios")
    print("População IBGE coletada:", df_pop.shape)
except Exception:
    print("[WARN] Nenhum município IBGE retornou dados válidos. Gerando dados simulados.")

    df_pop = pd.DataFrame({
        "municipio_id": mun_ids,
        "pop_density": np.random.uniform(50, 5000, size=len(mun_ids))
    })

df_pop.to_csv("data/raw/pop_density.csv", index=False)

KeyError: 'Localidade_id'

In [49]:
#Conclusão
print("Coleta de dados concluída com sucesso.")
print("Arquivos salvos em data/raw/:")
print("- dengue.csv")
print("- clima_inmet.csv")
print("- pop_density.csv")

Coleta de dados concluída com sucesso.
Arquivos salvos em data/raw/:
- dengue.csv
- clima_inmet.csv
- pop_density.csv


In [39]:
#Arquivos Gerados
#- data/raw/dengue.csv → base original do InfoDengue
# data/raw/clima_inmet.csv → dados climáticos simulados
# data/raw/pop_density.csv → densidade populacional simulada
